# Codificación de una arquitectura LLM

Los  LLM,  como  GPT  (que  significa  Transformador  Generativo  Preentrenado),  son  grandes  arquitecturas  de  redes  neuronales  profundas  diseñadas  para  generar  texto  nuevo,  una  palabra  (o  token)  a  la  vez. Sin  embargo,  a  pesar  de  su  tamaño,  la  arquitectura  del  modelo  es  menos  compleja  de  lo  que  se  podría  pensar,  ya  que  muchos  de  sus  componentes  se  repiten.

![Texto alternativo](./imgs/4.2.png)

En secciones anteriores se utilizaron dimensiones  de  incrustación  más  pequeñas  para  simplificar,  garantizando  que  los  conceptos  y  ejemplos  cupieran  cómodamente  en  una  sola  página.  Ahora,  se ampliará  el  tamaño  al  de  un  modelo  GPT2  pequeño,  específicamente  a  la  versión  más  pequeña  con  124  millones  de  parámetros,  como  se  describe  en  el  artículo  de  Radford  et  al. ,  "Los  modelos  de  lenguaje  son  aprendices  multitarea  no  supervisados".  Cabe  destacar  que,  si  bien  el  informe  original  menciona  117  millones  de  parámetros,  esto  se  corrigió  posteriormente.

En  el  contexto  del  aprendizaje  profundo  y  LLM  como  GPT,  el  término  "parámetros"  se  refiere  a  los  pesos  entrenables  del  modelo.  Estos  pesos  son,  en  esencia,  las  variables  internas  del  modelo  que  se  ajustan  y  optimizan  durante  el  proceso  de  entrenamiento  para  minimizar  una  función  de  pérdida  específica.  Esta  optimización  permite  que  el  modelo  aprenda  de  los  datos  de  entrenamiento.

Por  ejemplo,  en  una  capa  de  red  neuronal  representada  por  una  matriz  (o  tensor)  de  pesos  de  2048  x  2048  dimensiones,  cada  elemento  de  esta  matriz  es  un  parámetro.  Dado  que  hay  2048  filas  y  2048  columnas,  el  número  total  de  parámetros  en  esta  capa  es  2048  multiplicado  por  2048,  lo  que  equivale  a  4 194 304  parámetros.

Ejemplo de configuración del pequeño modelo GPT-2 a traves del siguiente diccionario de Python.

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 1024,      # Context length: Número max de tokens de entrada que el modelo puede manejar
    "emb_dim": 768,       # Embedding dimension: tamaño de incrustación
    "n_heads": 12,        # Number of attention heads
    "n_layers": 12,       # Number of layers
    "drop_rate": 0.1,     # Dropout rate: caida del 10%
    "qkv_bias": False     # Query-Key-Value bias
}

Utilizando  la  configuración  anterior,  comenzaremos  este  capítulo  implementando  una  arquitectura  de  marcador  de  posición  GPT  (DummyGPTModel)  en  esta  sección.  Esto  nos  proporcionará  na  visión  general  de  cómo  todo  encaja  y  qué  otros  componentes  necesitamos  codificar  en  las  siguientes  secciones  para  ensamblar  la  arquitectura  completa  del  modelo  GPT.

![Texto alternativo](./imgs/4.3.png)

In [3]:
#Estructura GPT provisional DummyGPTModel

import torch
import torch.nn as nn
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]) #marcador de posición para TransformerBlock
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])          #marcador de posición para LayerNorm
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
class DummyTransformerBlock(nn.Module):                           #clase de marcador de posición simple que será reemplazado en un futuro
    def __init__(self, cfg):
        super().__init__()
    def forward(self, x):                                         #no se hace nada
        return x
class DummyLayerNorm(nn.Module):                                  #clase de marcador de posición simple que será reemplazado en un futuro
    def __init__(self, normalized_shape, eps=1e-5):               #los aprámetros son solo para imitar la interfaz de LayerNorm
        super().__init__()
    def forward(self, x):
        return x

La clase DummyGPTModel en este código define una versión simplificada de un modelo tipo GPT utilizando el módulo de redes neuronales de PyTorch (nn.Module).

La arquitectura del modelo en la clase DummyGPTModel consta de:

- Embeddings de tokens y embeddings posicionales,

- Dropout,

- Una serie de bloques transformadores (DummyTransformerBlock),

- Una normalización de capa final (DummyLayerNorm),

- Y una capa lineal de salida (out_head).

La configuración del modelo se pasa a través de un diccionario de Python, por ejemplo, el diccionario GPT_CONFIG_124M que creamos anteriormente.

El método forward describe el flujo de datos a través del modelo: calcula los embeddings de tokens y posicionales para los índices de entrada, aplica dropout, procesa los datos a través de los bloques transformadores, aplica la normalización y finalmente produce los logits mediante la capa lineal de salida.

El código anterior ya es funcional, como veremos más adelante en esta sección, después de preparar los

![Texto alternativo](./imgs/4.4.png)

Para implementar los pasos que se muestran, se tokeniza un lote que cosnta de dso entradas de texto para el modelo GPT utilizando el tokenizador tiktoken.

In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


A continuación se inicia una nueva instancia DUmmyGPTModel de 124 millones de parámetros la cual se alimentará de los parámetros del lote.

In [6]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape: ", logits.shape)
print(logits)

Output shape:  torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)


El  tensor  de  salida  tiene  dos  filas  que  corresponden  a  las  dos muestras  de  texto.  Cada  muestra  de  texto  consta  de  cuatro  tokens;  cada  token  es  un  vector  de  50.257  dimensiones,  que  coincide  con  el tamaño  del  vocabulario  del  tokenizador.

[Normalización de activaciones con normalización de capas](./2_normalizacion_activaciones_normalizando_capas.ipynb)